# Statistical Arbitrage for Sports Betting

Using: Live Sports Odds API
Documentation Link: https://the-odds-api.com/ 

This program will look for statistical arbitrage opportunities in the upcoming eight games across all sports.

In [186]:
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Importing Dependencies and Acquiring API Key

In [220]:
import requests
import xlsxwriter
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Border, Side, Font, Alignment, PatternFill, numbers

API_KEY = '0e2b7f7c7f464158e404ba8052b3e109'

### Defining Constants and Making API Call
- ``BET_SIZE`` is the monetary amount in USD that you are willing to make for each bet. For example, if you define ``BET_SIZE`` as 100, you are telling the program that you want to bet a total of 100 dollars USD for each arbitrage opportunity that the program finds

In [221]:
SPORT = 'basketball_nba' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

BET_SIZE = 50

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': '0e2b7f7c7f464158e404ba8052b3e109',
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
).json()


In [197]:
# odds_response

### Event Class
- Each ``Event`` object represents an indivudal sporting event
- The ``data`` parameter contains all of the odds data that is received from the API call

In [222]:
from contextlib import nullcontext
BOOKMAKER_INDEX = 0
NAME_INDEX = 1
ODDS_INDEX = 2
FIRST = 0

class Event:
    def __init__(self, data):
        if not isinstance(data, dict):
            raise ValueError("Input data must be a dictionary")
        if "sport_key" not in data or "id" not in data:
            raise ValueError("Input data must have 'sport_key' and 'id' keys")
        self.data = data
        self.sport_key = data['sport_key']
        self.id = data['id']
        self.total_arbitrage_percentage = 0
        self.best_odds = self.find_best_odds()
        self.num_outcomes = None
        self.expected_earnings = 0  # initialize expected_earnings to None
        
    def find_best_odds(self):
      try:
    # number of possible outcomes for a sporting event
        num_outcomes = len(self.data['bookmakers'][0]['markets'][0]['outcomes'])
        self.num_outcomes = num_outcomes

        # finding the best odds for each outcome in each event
        best_odds = [[None, None, float('-inf')] for _ in range(num_outcomes)]
        # [Bookmaker, Name, Price]

        bookmakers = self.data['bookmakers']
        for index, bookmaker in enumerate(bookmakers):

            # determing the odds offered by each bookmaker
            for outcome in range(num_outcomes):

                # determining if any of the bookmaker odds are better than the current best odds
                bookmaker_odds = float(bookmaker['markets'][0]['outcomes'][outcome]['price'])
                current_best_odds = best_odds[outcome][ODDS_INDEX]

                if bookmaker_odds > current_best_odds:
                    best_odds[outcome][BOOKMAKER_INDEX] = bookmaker['title']
                    best_odds[outcome][NAME_INDEX] = bookmaker['markets'][0]['outcomes'][outcome]['name']
                    best_odds[outcome][ODDS_INDEX] = bookmaker_odds

        self.best_odds = best_odds
        
        # Debugging print statements
        print(f"Best odds found for event {self.data['id']}: {best_odds}")
        
        return best_odds
      except (KeyError, IndexError) as e:
          self.num_outcomes = 0
          pass



    
    def arbitrage(self):
      if not self.best_odds:
          print(f"No best odds found for event {self.id}")
          self.total_arbitrage_percentage = 0
          return True
        
      total_arbitrage_percentage = 0
      for odds in self.best_odds:
          total_arbitrage_percentage += (1.0 / odds[ODDS_INDEX])
        
      self.total_arbitrage_percentage = total_arbitrage_percentage
      self.expected_earnings = (BET_SIZE / total_arbitrage_percentage) - BET_SIZE
    
      # if the sum of the reciprocals of the odds is less than 1, there is opportunity for arbitrage
      if total_arbitrage_percentage < 1:
          return True
      return False

    
    # converts decimal/European best odds to American best odds
    def convert_decimal_to_american(self):
        best_odds = self.best_odds
        for odds in best_odds:
            decimal = odds[ODDS_INDEX]
            if decimal >= 2:
                american = (decimal - 1) * 100
            elif decimal < 2:
                american = -100 / (decimal - 1)
            odds[ODDS_INDEX] = round(american, 2)
        return best_odds
     
    def calculate_arbitrage_bets(self):
        bet_amounts = []
        for outcome in range(self.num_outcomes):
            individual_arbitrage_percentage = 1 / self.best_odds[outcome][ODDS_INDEX]
            bet_amount = (BET_SIZE * individual_arbitrage_percentage) / self.total_arbitrage_percentage
            bet_amounts.append(round(bet_amount, 2))
        
        self.bet_amounts = bet_amounts
        return bet_amounts

### Parsing Events and Calculating Arbitrage Bets
- `BET_SIZE` is the amount of money in USD that you would like to bet across the outcomes of an event.
- This calculation will used unbiased arbitrage, where the profit is the same regardless of the outcome.

In [223]:
events = []
for data in odds_response:
    events.append(Event(data))

arbitrage_events = []
for event in events:
    event.find_best_odds()
    event.arbitrage()
    arbitrage_events.append(event)

for event in arbitrage_events:
    event.calculate_arbitrage_bets()

print("Arbitrage Events IDs: ", arbitrage_events)


Best odds found for event 72fdd7b26ad160d2d34bfb3740936f77: [['DraftKings', 'Charlotte Hornets', 5.0], ['DraftKings', 'Phoenix Suns', 1.2]]
Best odds found for event eb2a5570850ff4fc1971cbf4ee33f11f: [['BetMGM', 'Chicago Bulls', 1.48], ['Bovada', 'Detroit Pistons', 3.0]]
Best odds found for event d7ace9d9552848e175c24d67c052f6a8: [['BetUS', 'Boston Celtics', 1.43], ['DraftKings', 'Cleveland Cavaliers', 3.2]]
Best odds found for event 2e7c761114d98fcd4b2f7f4c32d8ddfa: [['DraftKings', 'Brooklyn Nets', 3.5], ['BetUS', 'New York Knicks', 1.38]]
Best odds found for event 3fcccbc04991e5b18c298abf14aab9ce: [['LowVig.ag', 'Miami Heat', 2.05], ['DraftKings', 'Philadelphia 76ers', 1.87]]
Best odds found for event b090c2acdc31674822eac28232affa92: [['DraftKings', 'Houston Rockets', 4.6], ['DraftKings', 'Memphis Grizzlies', 1.22]]
Best odds found for event aae4f2ed3c0289815f4bb2adf6db954f: [['MyBookie.ag', 'Los Angeles Lakers', 1.9], ['FanDuel', 'Oklahoma City Thunder', 2.0]]
Best odds found for e

### Creating Dataframe and Writing to Excel File

In [224]:
if len(arbitrage_events) > 0:
    MAX_OUTCOMES = max([event.num_outcomes for event in arbitrage_events])
else:
    MAX_OUTCOMES = 1
    
ARBITRAGE_EVENTS_COUNT = len(arbitrage_events)

my_columns = ['ID', 'Sport Key', 'Expected Earnings'] + list(np.array([[f'Bookmaker #{outcome}', f'Name #{outcome}', f'Odds #{outcome}', f'Amount to Buy #{outcome}'] for outcome in range(1, MAX_OUTCOMES + 1)]).flatten())
dataframe = pd.DataFrame(columns=my_columns)
print("Arbitrage Events IDs: ", arbitrage_events)


Arbitrage Events IDs:  [<__main__.Event object at 0x7fcd6c51d1c0>, <__main__.Event object at 0x7fcd6c51d100>, <__main__.Event object at 0x7fcd6c51dbe0>, <__main__.Event object at 0x7fcd6c51d3a0>, <__main__.Event object at 0x7fcd6c51db20>, <__main__.Event object at 0x7fcd6c51daf0>, <__main__.Event object at 0x7fcd6c51d580>, <__main__.Event object at 0x7fcd6c51dfa0>, <__main__.Event object at 0x7fcd6c51d190>]


In [225]:
for event in arbitrage_events:
    # print(event.best_odds)
    row = []
    row.append(event.id)
    row.append(event.sport_key)
    row.append(round(event.expected_earnings, 2))
    if event.best_odds is not None:
        for index, outcome in enumerate(event.best_odds):
            row.append(outcome[BOOKMAKER_INDEX])
            row.append(outcome[NAME_INDEX])
            row.append(outcome[ODDS_INDEX])
            row.append(event.bet_amounts[index])
    else:
        for i in range(MAX_OUTCOMES):
            row.extend([None, None, None, None])
    while len(row) < len(dataframe.columns):
        row.append(None)
    dataframe.loc[len(dataframe.index)] = row

In [229]:
writer = pd.ExcelWriter('bets2.xlsx')
dataframe.to_excel(writer, index=False)
writer.save()

### Formatting the Excel File

In [230]:
BLACK = '000000'
LIGHT_GREY = 'D6D6D6'
DARK_GREY = '9F9F9F'
RED = 'FEA0A0'
BLUE = 'A0CEFE'
YELLOW = 'FFE540'

COLORS = [RED, BLUE]

ID_COLUMN_FILL = PatternFill(fill_type='solid', start_color=DARK_GREY, end_color=DARK_GREY)
SPORT_KEY_COLUMN_FILL = PatternFill(fill_type='solid', start_color=LIGHT_GREY, end_color=LIGHT_GREY)
EXPECTED_EARNINGS_COLUMN_FILL = PatternFill(fill_type='solid', start_color=YELLOW, end_color=YELLOW)

CENTER_ALIGNMENT = Alignment(horizontal='center', vertical='bottom', indent=0)

TOP_ROW_BORDER = Border(bottom=Side(border_style='thick', color=BLACK))
NORMAL_ROW_BORDER = Border(top=Side(border_style='thin', color=LIGHT_GREY), bottom=Side(border_style='thin', color=DARK_GREY))

wb = load_workbook('bets2.xlsx')
ws = wb.active
ws.title = 'Upcoming'
# changing width
for col in range(1, 26):
    ws.column_dimensions[chr(col + 64)].width = 20

for cell in ws['A']:
    cell.fill = ID_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    
for cell in ws['B']:
    cell.fill = SPORT_KEY_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    
for cell in ws['C']:
    cell.fill = EXPECTED_EARNINGS_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    cell.number_format = numbers.BUILTIN_FORMATS[7]

START_INDEX = 'D'
for index in range(MAX_OUTCOMES):
    for col in ws[START_INDEX : chr(ord(START_INDEX) + 3)]:
        for cell in col:
            color = COLORS[int(index % 2)]
            cell.fill = PatternFill(fill_type='solid', start_color=color, end_color=color)
            cell.alignment = CENTER_ALIGNMENT
            if cell.column % 4 == 3:
                cell.number_format = numbers.BUILTIN_FORMATS[7]
            
    START_INDEX = chr(ord(START_INDEX) + 4)

for cell in ws['1']:
    cell.border = TOP_ROW_BORDER

for row in range(2, ARBITRAGE_EVENTS_COUNT + 2):
    for cell in ws[str(row)]:
        cell.border = NORMAL_ROW_BORDER
    
wb.save('upcoming_events_bets2.xlsx')